https://www.kaggle.com/code/mnassrib/titanic-logistic-regression-with-python

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn import preprocessing
import matplotlib.pyplot as plt

plt.rc("font", size=14)

In [6]:
import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)

In [7]:
import warnings
warnings.simplefilter(action='ignore')